In [12]:
import pandas as pd
import seaborn as sn
import os
import glob

In [13]:
df_climate = pd.read_csv('climate_data_raw.csv')

script_directory = os.path.dirname(os.path.abspath('geo_delays_1.csv'))

# Use glob to find all .csv files in the directory containing 'bus_delay_fuzz'
csv_files = glob.glob(os.path.join(script_directory, '*geo_delays*.csv'))

# Initialize an empty list to store individual DataFrames
dfs = []

# Read and append each DataFrame to the list
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
df_bus_delays = pd.concat(dfs, ignore_index=True)

In [14]:
df_bus_delays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588589 entries, 0 to 588588
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           588589 non-null  int64  
 1   Route                588511 non-null  object 
 2   Day                  588589 non-null  object 
 3   Location             587796 non-null  object 
 4   Incident             587654 non-null  object 
 5   Min Delay            551142 non-null  float64
 6   Min Gap              556190 non-null  float64
 7   Vehicle              518753 non-null  float64
 8   DateTime             577298 non-null  object 
 9   copy                 587769 non-null  object 
 10  Unnamed: 0_x         198030 non-null  float64
 11  fuzz_locations       198030 non-null  object 
 12  fuzzed               198030 non-null  object 
 13  Unnamed: 0_y         198030 non-null  float64
 14  INTERSECTION_DESC    198030 non-null  object 
 15  CLASSIFICATION_DE

In [69]:
df_climate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121135 entries, 0 to 121134
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         121135 non-null  int64  
 1   LOCAL_HOUR         121135 non-null  int64  
 2   WIND_DIRECTION     117371 non-null  float64
 3   WINDCHILL          22114 non-null   float64
 4   PRECIP_AMOUNT      82143 non-null   float64
 5   HUMIDEX            18858 non-null   float64
 6   RELATIVE_HUMIDITY  114035 non-null  float64
 7   LOCAL_YEAR         121135 non-null  int64  
 8   TEMP_FLAG          422 non-null     object 
 9   UTC_MONTH          121135 non-null  int64  
 10  UTC_DAY            121135 non-null  int64  
 11  LOCAL_DATE         121135 non-null  object 
 12  STATION_PRESSURE   120692 non-null  float64
 13  TEMP               120712 non-null  float64
 14  UTC_YEAR           121135 non-null  int64  
 15  WIND_SPEED         120990 non-null  float64
 16  LO

In [15]:
# Convert 'LOCAL_DATE' and 'DateTime' columns to datetime objects
df_climate['LOCAL_DATE'] = pd.to_datetime(df_climate['LOCAL_DATE'])
df_bus_delays['DateTime'] = pd.to_datetime(df_bus_delays['DateTime'])

# Find the earliest timestamp in df_bus_delays
earliest_timestamp = df_bus_delays['DateTime'].min()

# Filter out rows in df_climate before the earliest timestamp
df_climate = df_climate[df_climate['LOCAL_DATE'] >= earliest_timestamp]
df_bus_delays = df_bus_delays.dropna(subset=['DateTime'])

df_bus_delays['temp'] = df_bus_delays['DateTime'].dt.round('H')

# Sort both dataframes based on the timestamp
df_climate = df_climate.sort_values(by='LOCAL_DATE')
df_bus_delays = df_bus_delays.sort_values(by='temp')
df_bus_delays = df_bus_delays.drop(columns=['Unnamed: 0_x'])

In [11]:
df_bus_delays.head()

,Unnamed: 0,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime,copy,...,INTERSECTION_DESC,CLASSIFICATION_DESC,geometry,lat,lon,closest_stop,stop_lon,stop_lat,analysis,temp
0,0,95.0,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,2014-01-01 00:23:00,yorkmillsstation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yorkmillsstation,2014-01-01 00:00:00
1,1,102.0,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,2014-01-01 00:55:00,entirerunforroute,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,entirerunforroute,2014-01-01 01:00:00
2,2,54.0,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,2014-01-01 01:28:00,lawrence&warden,...,lawrenceavee&wardenave,Major-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.29...",43.745377,-79.294730,Lawrence Ave East at Warden Ave,-79.294565,43.745551,lawrenceavee&wardenave,2014-01-01 01:00:00
3,3,112.0,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,2014-01-01 01:30:00,kiplingstation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kiplingstation,2014-01-01 02:00:00
4,4,24.0,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,2014-01-01 01:37:00,ellesmere&victoriapark,...,ellesmererd,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.27...",43.766165,-79.279371,Ellesmere Rd at West Service Rd East Side,-79.278977,43.766111,ellesmererd,2014-01-01 02:00:00


In [16]:
# Merge dataframes based on the closest timestamp
merged_df = pd.merge_asof(
    df_bus_delays,
    df_climate,
    left_on='temp',  # Use 'DateTime' as the key for the left dataframe
    right_on='LOCAL_DATE',  # Use 'LOCAL_DATE' as the key for the right dataframe
    direction='nearest'
)

# Drop duplicate columns, keeping only one of the timestamp columns
#merged_df = merged_df.drop(columns='LOCAL_DATE')

In [5]:
merged_df['time_deviation'] = merged_df['temp'] - merged_df['LOCAL_DATE']

In [83]:
merged_df['time_deviation'].max()

Timedelta('0 days 04:00:00')

In [17]:
num_chunks = 10
chunk_size = len(merged_df) // num_chunks

# Save each chunk into a separate CSV file
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i < num_chunks - 1 else len(merged_df)
    
    chunk_df = merged_df.iloc[start_idx:end_idx]
    
    # Save the chunk to a CSV file with a sequential number
    chunk_df.to_csv(f'geo_weather_{i + 1}.csv', index=False)

In [18]:
merged_df.head()

,Unnamed: 0_x,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime,copy,...,STATION_PRESSURE,TEMP,UTC_YEAR,WIND_SPEED,LOCAL_DAY,DEW_POINT_TEMP,LOCAL_MONTH,UTC_DATE,VISIBILITY,weather
0,0,95.0,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,2014-01-01 00:23:00,yorkmillsstation,...,101.56,-8.2,2014,39.0,1,-14.5,1,2014-01-01T06:00:00,16.1,dry
1,1,102.0,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,2014-01-01 00:55:00,entirerunforroute,...,101.56,-8.2,2014,39.0,1,-14.5,1,2014-01-01T06:00:00,16.1,dry
2,2,54.0,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,2014-01-01 01:28:00,lawrence&warden,...,101.56,-8.2,2014,39.0,1,-14.5,1,2014-01-01T06:00:00,16.1,dry
3,3,112.0,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,2014-01-01 01:30:00,kiplingstation,...,101.62,-8.6,2014,30.0,1,-14.4,1,2014-01-01T07:00:00,16.1,dry
4,4,24.0,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,2014-01-01 01:37:00,ellesmere&victoriapark,...,101.62,-8.6,2014,30.0,1,-14.4,1,2014-01-01T07:00:00,16.1,dry
